In [ ]:
import numpy as np
import xarray as xr

import datetime as dt
import pandas as pd

import matplotlib.pyplot as plt

import altair as alt
alt.data_transformers.enable('json')

from sublimpy import utils
import glob
import pytz
from scipy.signal import welch, csd
from scipy.stats import chi2
from process_fast_data.fast_data_calculate_spectra_nomrd import calculate_mrd_for_df, fast_data_files_to_dataframe

# Open SOS slow data

In [ ]:
sos_df = pd.read_parquet("../paper1/process_slow_data/tidy_df_20221101_20230619_planar_fit_multiplane_q7_flags9000_pf10.parquet")
sos_df_localtime = utils.modify_df_timezone( sos_df, 'UTC', 'US/Mountain')

## Plot wind directions for case studies

In [ ]:
def wind_chart(data):
    return alt.Chart(data).mark_line().encode(        
        alt.X('time:T'),
        alt.Y('value:Q'),
        alt.Color('height:O').scale(scheme='rainbow')
    ).properties(height=150)
def vertical_lines_chart(data):
    return alt.Chart(
        data
    ).mark_rule(color='red', strokeDash=[4, 2]).encode(
        alt.X('time:T')
    )


sos_dir_df = sos_df_localtime[sos_df_localtime.variable.isin([
        'dir_3m_c', 'dir_10m_c', 'dir_20m_c'
    ])].set_index('time')

(
    (wind_chart(sos_dir_df.loc['20230409'].reset_index())   + vertical_lines_chart(pd.DataFrame({'time': ['2023-04-09T15:00:00', '2023-04-09T18:00:00']}))) 
    | (wind_chart(sos_dir_df.loc['20230418'].reset_index()) + vertical_lines_chart(pd.DataFrame({'time': ['2023-04-18T15:00:00', '2023-04-18T18:00:00']}))) 
    | (wind_chart(sos_dir_df.loc['20221221'].reset_index()) + vertical_lines_chart(pd.DataFrame({'time': ['2022-12-21T15:00:00', '2022-12-21T18:00:00']}))) 
    | (wind_chart(sos_dir_df.loc['20230304'].reset_index()) + vertical_lines_chart(pd.DataFrame({'time': ['2023-03-04T15:00:00', '2023-03-04T18:00:00']})))
)

# Open SOS fast data

In [ ]:
apr18_df = fast_data_files_to_dataframe([
    # '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230418_21.nc',
    # '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230418_22.nc',
    # '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230418_23.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230419_01.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230419_02.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20230419_03.nc',
],  rotation='double')

apr18_df = utils.modify_df_timezone(apr18_df, pytz.UTC, "US/Mountain")

# Spectra across two case study days - Apr 18 and Dec 21

## Apr 18

### u'w' cospectra - all ECs

In [ ]:
apr18_u_w_3m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_c', 'w_3m_c', shift = 2000, parallelism=8)
apr18_u_w_3m_d_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_d', 'w_3m_d', shift = 2000, parallelism=8)
apr18_u_w_3m_ue_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_ue', 'w_3m_ue', shift = 2000, parallelism=8)
apr18_u_w_3m_uw_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_uw', 'w_3m_uw', shift = 2000, parallelism=8)

apr18_u_w_10m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_c', 'w_10m_c', shift = 2000, parallelism=8)
apr18_u_w_10m_d_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_d', 'w_10m_d', shift = 2000, parallelism=8)
apr18_u_w_10m_ue_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_ue', 'w_10m_ue', shift = 2000, parallelism=8)
apr18_u_w_10m_uw_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_uw', 'w_10m_uw', shift = 2000, parallelism=8)

apr18_u_w_5m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_5m_c', 'w_5m_c', shift = 2000, parallelism=8)
apr18_u_w_15m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_15m_c', 'w_15m_c', shift = 2000, parallelism=8)
apr18_u_w_20m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_20m_c', 'w_20m_c', shift = 2000, parallelism=8)

In [ ]:
base = alt.Chart().encode(
    alt.X('tau:Q').scale(type='log').title('tau (s)'),
)
def mrd_plot(data, title=''):
    return alt.layer(
        base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q')),
        base.mark_line().encode(alt.Y('mean(Co):Q')),
        data=data
    ).properties(width=200, height=200, title=title)

In [40]:
(
    (
        mrd_plot(apr18_u_w_3m_c_mrd_df, "u'w', 3m-C") | mrd_plot(apr18_u_w_3m_d_mrd_df, "u'w', 3m-D") 
        | mrd_plot(apr18_u_w_3m_ue_mrd_df, "u'w', 3m-UE") | mrd_plot(apr18_u_w_3m_uw_mrd_df, "u'w', 3m-UW")
    ).resolve_scale(y='shared') & (
        mrd_plot(apr18_u_w_10m_c_mrd_df, "u'w', 10m-C") | mrd_plot(apr18_u_w_10m_d_mrd_df, "u'w', 10m-D") 
        | mrd_plot(apr18_u_w_10m_ue_mrd_df, "u'w', 10m-UE") | mrd_plot(apr18_u_w_10m_uw_mrd_df, "u'w', 10m-UW")
    ).resolve_scale(y='shared') & (
        mrd_plot(apr18_u_w_5m_c_mrd_df, "u'w', 5m-C") |  mrd_plot(apr18_u_w_15m_c_mrd_df, "u'w', 15m-C") |  mrd_plot(apr18_u_w_20m_c_mrd_df, "u'w', 20m-C")
    ).resolve_scale(y='shared')
).resolve_scale(y='shared')

alt.VConcatChart(...)

### Isotropic stress cospectra - tower c 3 and 10m

In [33]:
apr18_u_u_3m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_3m_c', 'u_3m_c', shift = 2000, parallelism=8)
apr18_v_v_3m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'v_3m_c', 'v_3m_c', shift = 2000, parallelism=8)
apr18_w_w_3m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'w_3m_c', 'w_3m_c', shift = 2000, parallelism=8)

apr18_u_u_10m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'u_10m_c', 'u_10m_c', shift = 2000, parallelism=8)
apr18_v_v_10m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'v_10m_c', 'v_10m_c', shift = 2000, parallelism=8)
apr18_w_w_10m_c_mrd_df = calculate_mrd_for_df(apr18_df, 'w_10m_c', 'w_10m_c', shift = 2000, parallelism=8)

100%|██████████| 42/42 [00:16<00:00,  2.62it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.92it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.96it/s]


...

100%|██████████| 42/42 [00:13<00:00,  3.00it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.99it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.86it/s]


In [43]:
((
    mrd_plot(apr18_u_u_3m_c_mrd_df, "u'u', 3m-C") | mrd_plot(apr18_v_v_3m_c_mrd_df, "v'v', 3m-C") 
).resolve_scale(y='shared') & (
    mrd_plot(apr18_u_u_10m_c_mrd_df, "u'u', 10-C") | mrd_plot(apr18_v_v_10m_c_mrd_df, "v'v', 3m-C")
).resolve_scale(y='shared')).resolve_scale(y='shared') | (
    mrd_plot(apr18_w_w_3m_c_mrd_df, "w'w', 3m-C") & mrd_plot(apr18_w_w_10m_c_mrd_df, "w'w', 10m-C")
).resolve_scale(y='shared')

alt.HConcatChart(...)

## Dec 21

In [44]:
dec21_df = fast_data_files_to_dataframe([
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20221221_21.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20221221_22.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_20221221_23.nc',
],  rotation='double')
dec21_df = utils.modify_df_timezone(dec21_df, pytz.UTC, "US/Mountain")

mean u, v, w after first rotation  u_2m_c    5.325658e+00
v_2m_c   -4.837601e-07
w_2m_c    8.059380e-02
dtype: float32
mean u, v, w after first rotation  u_3m_c    5.669217e+00
v_3m_c    1.130281e-07
w_3m_c    5.993508e-02
dtype: float32
mean u, v, w after first rotation  u_5m_c    6.094958e+00
v_5m_c   -1.537182e-07
w_5m_c    6.208957e-02
dtype: float32
mean u, v, w after first rotation  u_10m_c    6.465034e+00
v_10m_c    5.470558e-07
w_10m_c   -6.427024e-03
dtype: float32
mean u, v, w after first rotation  u_15m_c    6.803667e+00
v_15m_c   -2.961335e-07
w_15m_c    5.895937e-02
dtype: float32
mean u, v, w after first rotation  u_20m_c    7.077481e+00
v_20m_c   -2.215350e-07
w_20m_c    2.049043e-02
dtype: float32
mean u, v, w after first rotation  u_3m_uw    5.593379e+00
v_3m_uw   -5.900065e-07
w_3m_uw    1.943769e-02
dtype: float32
mean u, v, w after first rotation  u_10m_uw    6.501176e+00
v_10m_uw    1.548485e-07
w_10m_uw   -1.366047e-02
dtype: float32
mean u, v, w after first rotat

### u'w' cospectra - all ECs

In [45]:
dec21_u_w_3m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'u_3m_c', 'w_3m_c', shift = 2000, parallelism=8)
dec21_u_w_3m_d_mrd_df = calculate_mrd_for_df(dec21_df, 'u_3m_d', 'w_3m_d', shift = 2000, parallelism=8)
dec21_u_w_3m_ue_mrd_df = calculate_mrd_for_df(dec21_df, 'u_3m_ue', 'w_3m_ue', shift = 2000, parallelism=8)
dec21_u_w_3m_uw_mrd_df = calculate_mrd_for_df(dec21_df, 'u_3m_uw', 'w_3m_uw', shift = 2000, parallelism=8)

dec21_u_w_10m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'u_10m_c', 'w_10m_c', shift = 2000, parallelism=8)
dec21_u_w_10m_d_mrd_df = calculate_mrd_for_df(dec21_df, 'u_10m_d', 'w_10m_d', shift = 2000, parallelism=8)
dec21_u_w_10m_ue_mrd_df = calculate_mrd_for_df(dec21_df, 'u_10m_ue', 'w_10m_ue', shift = 2000, parallelism=8)
dec21_u_w_10m_uw_mrd_df = calculate_mrd_for_df(dec21_df, 'u_10m_uw', 'w_10m_uw', shift = 2000, parallelism=8)

dec21_u_w_5m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'u_5m_c', 'w_5m_c', shift = 2000, parallelism=8)
dec21_u_w_15m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'u_15m_c', 'w_15m_c', shift = 2000, parallelism=8)
dec21_u_w_20m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'u_20m_c', 'w_20m_c', shift = 2000, parallelism=8)

Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:16<00:00,  2.61it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.99it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


...

100%|██████████| 42/42 [00:13<00:00,  3.04it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.05it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.06it/s]


In [56]:
def mrd_plot(data, title='', color=None):
    base = alt.Chart().encode(
        alt.X('tau:Q').scale(type='log').title('tau (s)'),
    )
    if color:
        band = base.mark_errorband(extent='iqr', color=color).encode(alt.Y('Co:Q'))
        line = base.mark_line(color=color).encode(alt.Y('mean(Co):Q'))
    else:
        band = base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q'))
        line = base.mark_line().encode(alt.Y('mean(Co):Q'))
    return alt.layer(
        band, line, data=data
    ).properties(width=200, height=200, title=title)

In [57]:
(
    (
        mrd_plot(dec21_u_w_3m_c_mrd_df, "u'w', 3m-C") | mrd_plot(dec21_u_w_3m_d_mrd_df, "u'w', 3m-D") 
        | mrd_plot(dec21_u_w_3m_ue_mrd_df, "u'w', 3m-UE") | mrd_plot(dec21_u_w_3m_uw_mrd_df, "u'w', 3m-UW")
    ).resolve_scale(y='shared') & (
        mrd_plot(dec21_u_w_10m_c_mrd_df, "u'w', 10m-C") | mrd_plot(dec21_u_w_10m_d_mrd_df, "u'w', 10m-D") 
        | mrd_plot(dec21_u_w_10m_ue_mrd_df, "u'w', 10m-UE") | mrd_plot(dec21_u_w_10m_uw_mrd_df, "u'w', 10m-UW")
    ).resolve_scale(y='shared') & (
        mrd_plot(dec21_u_w_5m_c_mrd_df, "u'w', 5m-C") |  mrd_plot(dec21_u_w_15m_c_mrd_df, "u'w', 15m-C") |  mrd_plot(dec21_u_w_20m_c_mrd_df, "u'w', 20m-C")
    ).resolve_scale(y='shared')
).resolve_scale(y='shared')

alt.VConcatChart(...)

In [74]:
(mrd_plot(dec21_u_w_3m_c_mrd_df, "u'w', 3m-C") + mrd_plot(apr18_u_w_3m_c_mrd_df, "u'w', 3m-C", color='#ff7f0e') + mrd_plot(apr18_u_w_3m_d_mrd_df, "u'w', 3m-C", color='#ff7f0e') + mrd_plot(apr18_u_w_3m_ue_mrd_df, "u'w', 3m-C", color='#ff7f0e') + mrd_plot(apr18_u_w_3m_uw_mrd_df, "u'w', 3m-C", color='#ff7f0e')) |\
(mrd_plot(dec21_u_w_10m_c_mrd_df, "u'w', 10m-C") + mrd_plot(apr18_u_w_10m_c_mrd_df, "u'w', 10m-C", color='#ff7f0e')) |\
(mrd_plot(dec21_u_w_5m_c_mrd_df, "u'w', 5m-C") + mrd_plot(apr18_u_w_5m_c_mrd_df, "u'w', 5m-C", color='#ff7f0e'))

alt.HConcatChart(...)

### Isotropic stress cospectra - tower c 3 and 10m

In [48]:
dec21_u_u_3m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'u_3m_c', 'u_3m_c', shift = 2000, parallelism=8)
dec21_v_v_3m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'v_3m_c', 'v_3m_c', shift = 2000, parallelism=8)
dec21_w_w_3m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'w_3m_c', 'w_3m_c', shift = 2000, parallelism=8)

dec21_u_u_10m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'u_10m_c', 'u_10m_c', shift = 2000, parallelism=8)
dec21_v_v_10m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'v_10m_c', 'v_10m_c', shift = 2000, parallelism=8)
dec21_w_w_10m_c_mrd_df = calculate_mrd_for_df(dec21_df, 'w_10m_c', 'w_10m_c', shift = 2000, parallelism=8)

Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:14<00:00,  2.99it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.05it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


...

100%|██████████| 42/42 [00:13<00:00,  3.05it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.03it/s]


Got data of length 216000. using M = 17
Timestep of fast data is: 50.0 ms
Dataset of length 216000, with M = 17, permits 42 iterations.


100%|██████████| 42/42 [00:13<00:00,  3.04it/s]


In [49]:
((
    mrd_plot(dec21_u_u_3m_c_mrd_df, "u'u', 3m-C") | mrd_plot(dec21_v_v_3m_c_mrd_df, "v'v', 3m-C") 
).resolve_scale(y='shared') & (
    mrd_plot(dec21_u_u_10m_c_mrd_df, "u'u', 10-C") | mrd_plot(dec21_v_v_10m_c_mrd_df, "v'v', 3m-C")
).resolve_scale(y='shared')).resolve_scale(y='shared') | (
    mrd_plot(dec21_w_w_3m_c_mrd_df, "w'w', 3m-C") & mrd_plot(dec21_w_w_10m_c_mrd_df, "w'w', 10m-C")
).resolve_scale(y='shared')

alt.HConcatChart(...)

# Spectra across two months of data

Open up all the nomrd files. Calculate a "median time" for joining with synoptic wind measurements

In [216]:
nomrd_df = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/")
nomrd_df['median_time'] = nomrd_df.start_time + 0.5*(nomrd_df.end_time - nomrd_df.start_time)
nomrd_df['median_time_rounded'] = nomrd_df.median_time.dt.round('30min')
nomrd_df

,tau,Co,std,iteration,start_time,end_time,covariance,loc,median_time,median_time_rounded
0,0.1,-0.002316,4.401546e-02,0,2022-12-21 14:00:00,2022-12-21 15:49:13.550,u_w_,3m_c,2022-12-21 14:54:36.775,2022-12-21 15:00:00
1,0.2,-0.004666,4.751782e-02,0,2022-12-21 14:00:00,2022-12-21 15:49:13.550,u_w_,3m_c,2022-12-21 14:54:36.775,2022-12-21 15:00:00
2,0.4,-0.007815,5.590110e-02,0,2022-12-21 14:00:00,2022-12-21 15:49:13.550,u_w_,3m_c,2022-12-21 14:54:36.775,2022-12-21 15:00:00
3,0.8,-0.011504,6.878115e-02,0,2022-12-21 14:00:00,2022-12-21 15:49:13.550,u_w_,3m_c,2022-12-21 14:54:36.775,2022-12-21 15:00:00
4,1.6,-0.011965,6.791920e-02,0,2022-12-21 14:00:00,2022-12-21 15:49:13.550,u_w_,3m_c,2022-12-21 14:54:36.775,2022-12-21 15:00:00
...,...,...,...,...,...,...,...,...,...,...
12,409.6,0.000337,1.142486e-03,41,2023-04-30 16:08:20,2023-04-30 17:57:33.550,w_h2o_,10m_d,2023-04-30 17:02:56.775,2023-04-30 17:00:00
13,819.2,0.001552,1.921631e-03,41,2023-04-30 16:08:20,2023-04-30 17:57:33.550,w_h2o_,10m_d,2023-04-30 17:02:56.775,2023-04-30 17:00:00
14,1638.4,0.000481,4.260532e-04,41,2023-04-30 16:08:20,2023-04-30 17:57:33.550,w_h2o_,10m_d,2023-04-30 17:02:56.775,2023-04-30 17:00:00
15,3276.8,-0.000060,7.827031e-06,41,2023-04-30 16:08:20,2023-04-30 17:57:33.550,w_h2o_,10m_d,2023-04-30 17:02:56.775,2023-04-30 17:00:00


Use doppler lidar data to identify along-valley/cross-valley synoptic flows

In [217]:
cross_valley_wind_times = pd.read_csv('cross_valley_wind_times.csv').drop(columns='Unnamed: 0').rename(columns={'0':'time'}).set_index('time').assign(
    wind_dir='cross-valley'
)
cross_valley_wind_times.index = pd.to_datetime(cross_valley_wind_times.index)

along_valley_wind_times = pd.read_csv('along_valley_wind_times.csv').drop(columns='Unnamed: 0').rename(columns={'0':'time'}).set_index('time').assign(
    wind_dir='along-valley'
)
along_valley_wind_times.index = pd.to_datetime(along_valley_wind_times.index)

winds_ge10_times =      pd.read_csv('winds_ge10_times.csv').drop(columns='Unnamed: 0').set_index('time').assign(wind_speed='> 10')
winds_ge10_times.index = pd.to_datetime(winds_ge10_times.index)
winds_le10_times =      pd.read_csv('winds_le10_times.csv').drop(columns='Unnamed: 0').set_index('time').assign(wind_speed='<= 10')
winds_le10_times.index = pd.to_datetime(winds_le10_times.index)

Join the datasets

In [218]:
nomrd_df_synopticwinds = pd.concat([
    nomrd_df.set_index('median_time_rounded').join(cross_valley_wind_times, how='inner'),
    nomrd_df.set_index('median_time_rounded').join(along_valley_wind_times, how='inner')
])
nomrd_df_synopticwinds = nomrd_df_synopticwinds.join(pd.concat([ winds_le10_times, winds_ge10_times]), how='inner')

In [220]:
nomrd_df_synopticwinds.wind_speed.unique()

array(['<= 10', '> 10'], dtype=object)

In [227]:
(alt.Chart(
    nomrd_df_synopticwinds.query("covariance == 'u_w_'").query("loc == '3m_c'")
).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q'),
    alt.Color('wind_dir:N'),
    alt.Row('wind_speed')
).properties(width=200, height=200, title = "u'w', 3m-c") | alt.Chart(
    nomrd_df_synopticwinds.query("covariance == 'u_w_'").query("loc == '5m_c'")
).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q'),
    alt.Color('wind_dir:N'),
    alt.Row('wind_speed')
).properties(width=200, height=200, title = "u'w', 5m-c")).resolve_scale(x='shared', y='shared')

alt.HConcatChart(...)

In [229]:
(alt.Chart(
    nomrd_df_synopticwinds.query("covariance == 'v_w_'").query("loc == '3m_c'")
).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q').scale(domain = [-0.001, 0.005], clamp=True),
    alt.Color('wind_dir:N'),
    alt.Row('wind_speed')
).properties(width=200, height=200, title = "u'w', 3m-c") | alt.Chart(
    nomrd_df_synopticwinds.query("covariance == 'v_w_'").query("loc == '5m_c'")
).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q').scale(domain = [-0.001, 0.005], clamp=True),
    alt.Color('wind_dir:N'),
    alt.Row('wind_speed')
).properties(width=200, height=200, title = "u'w', 5m-c")).resolve_scale(x='shared', y='shared')

alt.HConcatChart(...)

# Load preprocessed spectra, compare across towers

In [ ]:
spectra_mar04 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2023-03-04 14:00:00_2023-03-04 16:59:59.950000.parquet")
spectra_mar04['height'] = spectra_mar04['loc'].str.split('m').str[0].astype('int')
spectra_mar04['tower'] = spectra_mar04['loc'].str.split('m_').str[1]
spectra_mar04.covariance.unique()

In [ ]:
(alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_mar04.query("covariance == 'w_tc_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'w_h2o_'").query("height == 3")
).facet('height') )

In [ ]:
(alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_mar04.query("covariance == 'u_u_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'v_v_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'w_w_'").query("height == 3")
).facet('height'))

In [ ]:
alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_mar04.query("covariance == 'u_w_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'v_w_'").query("height == 3")
).facet('height') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(v'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(v'w')")),
    data = spectra_mar04.query("covariance == 'u_v_'").query("height == 3")
).facet('height')

In [ ]:
spectra_apr18 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2023-04-18 15:00:00_2023-04-18 17:59:59.950000.parquet")
spectra_apr18['height'] = spectra_apr18['loc'].str.split('m').str[0].astype('int')
spectra_apr18['tower'] = spectra_apr18['loc'].str.split('m_').str[1]
spectra_apr18.covariance.unique()

base = alt.Chart().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Color('tower'),
).properties(width=200, height=150)

(alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr18.query("covariance == 'u_w_'").query("height == 3")
).facet('height').properties(title='Apr 18') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_1221.query("covariance == 'u_w_'").query("height == 3")
).facet('height').properties(title='Dec 21') | alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr09.query("covariance == 'u_w_'").query("height == 3")
).facet('height').properties(title='Apr 09')).resolve_scale(y='shared', x='shared')

In [ ]:
apr18_df = prep_fast_data([
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_21.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_22.nc',
    '/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast_old/isfs_sos_qc_geo_tiltcor_hr_20230418_23.nc',
])

In [ ]:
len(apr18_df.set_index('time')['u_3m_c']), len(apr18_df.set_index('time')['u_3m_c'].dropna())

In [ ]:
len(apr18_df.set_index('time')['u_3m_d']), len(apr18_df.set_index('time')['u_3m_d'].dropna())

In [ ]:
len(apr18_df.set_index('time')['u_3m_uw']), len(apr18_df.set_index('time')['u_3m_uw'].dropna())

In [ ]:
len(apr18_df.set_index('time')['u_3m_ue']), len(apr18_df.set_index('time')['u_3m_ue'].dropna())

In [ ]:
alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr18.query("covariance == 'u_w_'").query("height == 3")
).facet('height').properties(title='Apr 18, 1500-1800')

In [ ]:
spectra_1221 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2022-12-21 14:00:00_2022-12-21 16:59:59.950000.parquet")
spectra_1221['height'] = spectra_1221['loc'].str.split('m').str[0].astype('int')
spectra_1221['tower'] = spectra_1221['loc'].str.split('m_').str[1]

spectra_apr18 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2023-04-18 15:00:00_2023-04-18 17:59:59.950000_s1000.parquet")
spectra_apr18['height'] = spectra_apr18['loc'].str.split('m').str[0].astype('int')
spectra_apr18['tower'] = spectra_apr18['loc'].str.split('m_').str[1]

spectra_apr09 = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/NOmrds/2023-04-09 15:00:00_2023-04-09 17:59:59.950000.parquet")
spectra_apr09['height'] = spectra_apr09['loc'].str.split('m').str[0].astype('int')
spectra_apr09['tower'] = spectra_apr09['loc'].str.split('m_').str[1]

base = alt.Chart().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Color('tower'),
).properties(width=200, height=150)

(alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr18.query("covariance == 'u_w_'").query("height == 10")
).facet('height').properties(title='Apr 18') |
alt.layer(
    base.mark_line().encode(alt.Y('mean(Co):Q').title("Co(u'w')")),
    base.mark_errorband(extent='iqr').encode(alt.Y('Co:Q').title("Co(u'w')")),
    data = spectra_apr09.query("covariance == 'u_w_'").query("height == 10")
).facet('height').properties(title='Apr 09')).resolve_scale(x='shared')

In [ ]:
alt.Chart(
    sos_df_localtime.query("measurement == 'wind speed'").query("tower == 'c'").set_index('time').sort_index().loc['20230409 1200': '20230409 1500'].reset_index()
).mark_line().encode(
    alt.X('value:Q'),
    alt.Y('height:Q'),
    alt.Facet('time:T'),
    order='height'
).properties(width=100, height=150)

In [ ]:
base = alt.Chart().mark_line().encode(
    alt.X('time:T'),
    # alt.Y('rolling_median:Q')
    alt.Y('value:Q')
).properties(width=200, height=150)
(
alt.layer(base, data=sos_df_localtime.query("variable == 'L_3m_c'").set_index('time').loc['20230418'].reset_index()) |\
alt.layer(base, data=sos_df_localtime.query("variable == 'L_3m_c'").set_index('time').loc['20221221'].reset_index()) |\
alt.layer(base, data=sos_df_localtime.query("variable == 'L_3m_c'").set_index('time').loc['20230409'].reset_index())
).resolve_scale(y='shared')

In [ ]:
base = alt.Chart().mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:N')
).properties(width=200, height=150)
sos_dir_data = sos_df_localtime[
    sos_df_localtime.variable.isin(['dir_3m_c', 'dir_20m_c'])
].set_index('time')

(
    alt.layer(base, data=sos_dir_data.loc['20230418'].reset_index()) |\
    alt.layer(base, data=sos_dir_data.loc['20221221'].reset_index()) |\
    alt.layer(base, data=sos_dir_data.loc['20230409'].reset_index()) |\
    alt.layer(base, data=sos_dir_data.loc['20230304'].reset_index()) 
).resolve_scale(y='shared')

# Get mrds for all days (1500-1800) with consistent winds

In [ ]:
import tqdm

In [ ]:
mrds_processed_data = {}

# for d in tqdm.tqdm(days_with_consistent_1500_1800_winds):
for d in tqdm.tqdm([d for d in days_with_consistent_1500_1800_winds if d.month == 4]):
    day_str = str(d).replace('-','')
    mrds_processed_data[day_str] = process_mrds_for_files([
        f'/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_{day_str}_21.nc',
        f'/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_{day_str}_22.nc',
        f'/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_{day_str}_23.nc',
    ])

In [ ]:
len(mrds_processed_data)